In [3]:
from pyscipopt import Model

m = Model("demo")
x = m.addVar("x", lb=0, ub=10)
m.setObjective(x, "maximize")
m.addCons(x <= 5)
m.optimize()
print("Optimal value:", m.getObjVal())

Optimal value:feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
 5.0
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +5.00000000000000e+00 (2 solutions)
Dual Bound         : +5.00000000000000e+00
Gap                : 0.00 %


In [4]:
from pyscipopt import Model, quicksum
import numpy as np

# Number of cities
n = 5

# Generate a random symmetric distance matrix
np.random.seed(42)
dist = np.random.randint(10, 100, size=(n, n))
for i in range(n):
    dist[i, i] = 0
    for j in range(i+1, n):
        dist[j, i] = dist[i, j]

print("Distance matrix:\n", dist)

# Create SCIP model
model = Model("TSP_MTZ")

# Binary variables: x[i,j] = 1 if path goes from i -> j
x = {}
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

# MTZ ordering variables: u[i] gives position of city i in tour
u = {}
for i in range(n):
    u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

# Objective: minimize travel distance
model.setObjective(
    quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
    "minimize"
)

# Each city has exactly one outgoing edge
for i in range(n):
    model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)

# Each city has exactly one incoming edge
for j in range(n):
    model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

# MTZ subtour elimination constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

# Solve the model
model.optimize()

# Print optimal solution
if model.getStatus() == "optimal":
    print(f"\nOptimal tour length = {model.getObjVal():.2f}")

    # Extract tour edges
    tour = []
    current = 0
    visited = {0}
    for _ in range(n):
        for j in range(n):
            if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                tour.append((current, j))
                current = j
                break
    print("Tour edges:", tour)


Distance matrix:
 [[ 0 24 81 70 30]
 [24  0 84 84 97]
 [81 84  0 62 11]
 [70 84 62  0 73]
 [30 97 11 73  0]]

Optimal tour length = 211.00
Tour edges: [(0, 1), (1, 3), (3, 2), (2, 4), (4, 0)]
presolving:
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 10 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 24 implications, 10 cliques
presolved problem has 24 variables (20 bin, 0 int, 0 impl, 4 cont) and 22 constraints
     10 constrai

In [ ]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import random

def solve_tsp_mtz(n, dist, instance_id):
    """
    Solve TSP with MTZ constraints using SCIP.
    Returns objective value and edges in the optimal tour.
    """
    model = Model(f"TSP_MTZ_{instance_id}")
    
    # Decision variables
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

    u = {}
    for i in range(n):
        u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

    # Objective
    model.setObjective(
        quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
        "minimize"
    )

    # Degree constraints
    for i in range(n):
        model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    # MTZ constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

    # Solve
    model.optimize()

    if model.getStatus() == "optimal":
        obj = model.getObjVal()
        # Extract tour edges
        edges = []
        current = 0
        visited = {0}
        for _ in range(n):
            for j in range(n):
                if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                    edges.append((current, j))
                    current = j
                    break
        return obj
    else:
        return None, None


def generate_instances(num_instances=100, min_cities=5, max_cities=10, seed=42):
    """
    Generate and solve multiple random TSP instances.
    Returns a pandas DataFrame with summary results.
    """
    random.seed(seed)
    np.random.seed(seed)

    results = []
    for inst in range(num_instances):
        n = random.randint(min_cities, max_cities)
        dist = np.random.randint(10, 100, size=(n, n))
        for i in range(n):
            dist[i, i] = 0
            for j in range(i+1, n):
                dist[j, i] = dist[i, j]

        obj = solve_tsp_mtz(n, dist, inst)
        if obj is not None:
            results.append({
                "instance": inst,
                "cities": n,
                "objective": obj
            })
            print(f"Instance {inst}: {n} cities, optimal cost = {obj:.2f}")
        else:
            print(f"Instance {inst}: solver failed")

    return pd.DataFrame(results)


# Example: generate 10 small TSP instances
df = generate_instances(num_instances=10, min_cities=5, max_cities=7)

print(df.head())


presolving:
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 14 clqs
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 14 upgd conss, 0 impls, 14 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 60 implications, 14 cliques
presolved problem has 48 variables (42 bin, 0 int, 0 impl, 6 cont) and 44 constraints
     14 constraints of type <setppc>
     30 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt 

In [ ]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE
import math

INT_TOL = 1e-6  # integrality tolerance

def _is_integral(model, vars_):
    """Check if current LP solution (at focused node) is integral on all integer vars."""
    for v in vars_:
        if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
            val = model.getSolVal(None, v)  # None = current LP solution
            if abs(val - round(val)) > INT_TOL:
                return False
    return True

class NodeTracker(Eventhdlr):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.active = {}       # node_id -> dict(status, depth, bound, lpfeas, integral)
        self.sols = []         # list of dicts: {'obj': float, 'values': {varname: val}}
        self.allvars = None

    def eventinit(self):
        # Capture a broad set of node/LP/solution events; ignore if not present in this build
        for et in [
            SCIP_EVENTTYPE.NODEADDED,
            SCIP_EVENTTYPE.NODEBRANCHED,
            SCIP_EVENTTYPE.NODEFOCUSED,
            SCIP_EVENTTYPE.NODEINFEASIBLE,
            SCIP_EVENTTYPE.LPSOLVED,
            SCIP_EVENTTYPE.BESTSOLFOUND,
        ]:
            try:
                self.model.catchEvent(et, self)
            except:  # noqa
                pass
        self.allvars = self.model.getVars()

    def _mark_active(self, node):
        try:
            nid = node.getNumber()
            depth = self.model.getDepth()
        except:
            return
        if nid not in self.active:
            self.active[nid] = {
                "status": "open",
                "depth": depth,
                "bound": None,
                "lpfeas": None,
                "integral": None,
            }

    def _close_node(self, node, reason):
        try:
            nid = node.getNumber()
        except:
            return
        if nid in self.active:
            self.active[nid]["status"] = reason

    def eventexec(self, event):
        et = event.getType()

        # Node created or branched -> ensure it's tracked as open
        if et in (SCIP_EVENTTYPE.NODEADDED, SCIP_EVENTTYPE.NODEBRANCHED):
            try:
                node = event.getNode()
            except:
                node = self.model.getCurrentNode()
            if node:
                self._mark_active(node)

        # When a node gets focus, ensure it's tracked
        elif et == SCIP_EVENTTYPE.NODEFOCUSED:
            node = self.model.getCurrentNode()
            if node:
                self._mark_active(node)

        # After the node's LP is solved, we can classify LP feasibility & integrality
        elif et == SCIP_EVENTTYPE.LPSOLVED:
            node = self.model.getCurrentNode()
            if not node:
                return
            self._mark_active(node)
            try:
                lb = self.model.getLPObjVal()  # LP relaxation objective at node
                lpsolstat = self.model.getLPSolstat()  # may exist; not all builds expose
                lpfeas = True
            except:
                # If we can't query LP status cleanly, assume feasible unless proven infeasible elsewhere
                lb = None
                lpfeas = True
            self.active[node.getNumber()]["bound"] = lb
            self.active[node.getNumber()]["lpfeas"] = lpfeas

            # Check integrality of the current LP solution
            integral = _is_integral(self.model, self.allvars)
            self.active[node.getNumber()]["integral"] = integral
            if integral:
                # We found an integer-feasible node → log its solution, but keep cutoff at +inf
                vals = {}
                for v in self.allvars:
                    val = self.model.getSolVal(None, v)
                    # save only integer vars & binaries for brevity
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = round(val)
                try:
                    obj = self.model.getLPObjVal()
                except:
                    obj = None
                self.sols.append({"obj": obj, "values": vals})
                # Do NOT allow this to enable bound-pruning; keep cutoff infinite
                try:
                    self.model.setObjlimit(float("inf"))
                    self.model.setRealParam("limits/upperobjlimit", float("inf"))
                except:
                    pass
                # Close the node as 'integer'
                self._close_node(node, "integer")

        # Explicit infeasible node event → close it
        elif et == SCIP_EVENTTYPE.NODEINFEASIBLE:
            try:
                node = event.getNode()
            except:
                node = self.model.getCurrentNode()
            if node:
                self._close_node(node, "infeasible")

        # Solver found a (better) solution globally. We copy it out but keep cutoff = +inf.
        elif et == SCIP_EVENTTYPE.BESTSOLFOUND:
            sol = self.model.getBestSol()
            if sol:
                obj = self.model.getSolObjVal(sol)
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = round(self.model.getSolVal(sol, v))
                self.sols.append({"obj": obj, "values": vals})
                try:
                    self.model.setObjlimit(float("inf"))
                    self.model.setRealParam("limits/upperobjlimit", float("inf"))
                except:
                    pass

def solve_ip_enumerate_like(build_model_fn, node_limit, time_limit=None):
    """
    Run B&B with best-bound pruning neutralized, stop after `node_limit`,
    and return integer-feasible node solutions + an 'active node list' approximation.

    Args:
      build_model_fn: () -> Model  # builds and returns a PySCIPOpt Model (already with vars/cons)
      node_limit: int              # stop after this many processed nodes
      time_limit: float|None       # optional wall-clock limit in seconds

    Returns: dict with
      - status, nodes_processed, nodes_remaining, solving_time
      - integer_solutions: list of {obj, values{var:val}}
      - active_nodes: list of node summaries still 'open' at stop
    """
    model = build_model_fn()

    # Neutralize best-bound pruning
    model.setObjlimit(float("inf"))
    try:
        model.setRealParam("limits/upperobjlimit", float("inf"))
    except:
        pass

    # Optional: reduce heuristics to avoid easy incumbents (not required, but closer to enumeration)
    for p in [
        "heuristics/underroot", "heuristics/trysol", "heuristics/rins",
        "heuristics/repair", "heuristics/actconsdiving", "heuristics/shiftandpropagate",
    ]:
        try:
            model.setIntParam(p, 0)
        except:
            pass

    # DFS-ish node selection to keep the tree “enumeration-like”
    try:
        model.setStringParam("nodeselection/strategy", "dfs")
    except:
        pass

    # Hard stopping rules
    model.setLongintParam("limits/nodes", int(node_limit))
    if time_limit is not None:
        model.setRealParam("limits/time", float(time_limit))

    # Attach tracker
    tracker = NodeTracker(model)
    model.includeEventhdlr(tracker, "NodeTracker", "Track active nodes and integer solutions")

    # Solve
    model.optimize()

    # Build 'active node list' = nodes still marked 'open'
    active_nodes = []
    for nid, info in tracker.active.items():
        if info["status"] == "open":
            active_nodes.append({
                "node_id": nid,
                "depth": info["depth"],
                "lp_bound": info["bound"],
                "lp_feasible": info["lpfeas"],
                "integral": info["integral"],  # should be False for truly active
            })

    result = {
        "status": model.getStatus(),
        "nodes_processed": model.getNNodes(),
        "nodes_remaining": model.getNRemainingNodes(),
        "solving_time": model.getSolvingTime(),
        "integer_solutions": tracker.sols,  # every integral node we saw
        "active_nodes": active_nodes,       # approximation from events
    }

    try:
        model.free()
    except:
        pass

    return result